In [31]:
import sys
sys.executable

'c:\\Users\\thaih\\Documents\\K22\\Nam_3\\HK1\\MachineLearning\\Project\\NeuralNetworkForClassification\\venv\\Scripts\\python.exe'

## Import necessary libraries

In [32]:
import numpy as np
import os
import pickle
import tarfile
import gzip
import matplotlib.pyplot as plt
plt.style.use('ggplot')

## Data collection

In [33]:
def unpickle(file):
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

def extract_cifar10(tar_gz_file, extract_to):
    """ Extract the cifar-10 tar.gz file """
    with tarfile.open(tar_gz_file, 'r:gz') as tar:
        tar.extractall(path=extract_to)

def load_cifar10_data(data_dir):
    # List to store training data
    train_X, train_Y = [], []
    
    # Load training batches (data_batch_1 to data_batch_5)
    for i in range(1, 6):
        batch_filename = os.path.join(data_dir, f'data_batch_{i}')
        batch_data = unpickle(batch_filename)
        train_X.append(batch_data[b'data'])
        train_Y.append(batch_data[b'labels'])
    
    # Concatenate the training data into a single array
    train_X = np.concatenate(train_X)
    train_Y = np.concatenate(train_Y)
    
    # Load the test batch (test_batch)
    test_batch_filename = os.path.join(data_dir, 'test_batch')
    test_data = unpickle(test_batch_filename)
    
    test_X = test_data[b'data']
    test_Y = test_data[b'labels']
    
    train_X = np.array(train_X)
    train_Y = np.array(train_Y)
    test_X = np.array(test_X)
    test_Y = np.array(test_Y)
    
    return train_X, train_Y, test_X, test_Y

# Path to the tar.gz file
tar_gz_file = 'cifar-10-python.tar.gz'

# Check if the extracted folder exists, otherwise extract it
extracted_folder = 'cifar-10-batches-py'
if not os.path.exists(extracted_folder):
    extract_cifar10(tar_gz_file, '.')

# Load the data from the extracted folder
train_X, train_Y, test_X, test_Y = load_cifar10_data(extracted_folder)

print(f'Shape of train_X: {train_X.shape}, shape of train_Y: {train_Y.shape}')
print(f'Shape of test_X: {test_X.shape}, shape of test_Y: {test_Y.shape}')


Shape of train_X: (50000, 3072), shape of train_Y: (50000,)
Shape of test_X: (10000, 3072), shape of test_Y: (10000,)


- **data**: a 10000x3072 numpy array of uint8s. 
  - Each row of the array stores a 32x32 colour image. 
  - The first 1024 entries contain the red channel values, the next 1024 the green, and the final 1024 the blue. 
  - The image is stored in row-major order, so that the first 32 entries of the array are the red channel values of the first row of the image.
- **labels**: a list of 10000 numbers in the range 0-9. The number at index i indicates the label of the ith image in the array data.

In [34]:
# train_X[0] is a flattened array of size 3072
image = train_X[0]

# Reshaping to a 32x32x3 image (32x32 pixels, 3 color channels)
image_reshaped = image.reshape(32, 32, 3)

red_channel_first_row = image_reshaped[0, :, 0]  # First row, Red channel
green_channel_first_row = image_reshaped[0, :, 1]  # First row, Green channel
blue_channel_first_row = image_reshaped[0, :, 2]  # First row, Blue channel

# Print the first row of each channel
print(red_channel_first_row)
print(green_channel_first_row)
print(blue_channel_first_row)
print(image.shape)
print(image)

[ 59  68 139 149 142 129 124 133 152 159 152   0  88 127 106 113 119 125
 131 131 133 122  49 129 113 106 124 122 130 127 130 118]
[ 43  98 145 131 144 137 139 136 163 158 148  18 120 126 101 109 109 127
 124 132 123  25  83 130 112 105 130 115 131 126 142 120]
[ 50 119 149 125 137 134 139 139 168 158  16  51 128 116 105 112 105 122
 121 133 119  16 110 121 112 128 127 120 139 127 130 109]
(3072,)
[ 59  43  50 ... 140  84  72]


## Data preprocessing

In [35]:
train_X = train_X / 255.0

## Feature engineering

In [37]:
def one_hot_encode(y, num_classes):
    return np.eye(num_classes)[y]

unique_classes_train = np.unique(train_Y)
num_classes = len(unique_classes_train)

# Encode training and testing labels
train_Y = one_hot_encode(train_Y, num_classes)
print("Train Y shape: ", train_Y.shape)

test_Y = one_hot_encode(test_Y, num_classes)
print("Test Y shape: ", test_Y.shape)


Train Y shape:  (50000, 10)
Test Y shape:  (10000, 10)


## Model Design

In [26]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import log_loss

In [40]:
mlp = MLPClassifier(
    hidden_layer_sizes=(2, 32),  # Two hidden layers with 50 and 30 neurons each
    activation='logistic',        # Sigmoid activation function
    solver='sgd',                 # Stochastic Gradient Descent optimizer
    learning_rate_init=0.01,      # Initial learning rate
    max_iter=500,                 # Maximum number of iterations
    random_state=42               # For reproducibility
)

mlp.fit(train_X, train_Y)

MLPClassifier(activation='logistic', hidden_layer_sizes=(2, 32),
              learning_rate_init=0.01, max_iter=500, random_state=42,
              solver='sgd')

In [41]:
y_pred = mlp.predict(test_X)
y_prob = mlp.predict_proba(test_X)
print(np.sum(y_prob, axis=1))
# cross_entropy_error = log_loss(test_Y, y_prob)

# print("Cross Entropy Error: ", cross_entropy_error)

[1.12623896 1.14729643 1.12623896 ... 1.00089061 1.00089061 1.00089061]
